In [ ]:
# requests for downloading things off the internet, in this case the CSV files
# pandas we know is good for handling data (could do all this without it but just makes things easier)
# re is the 'regular expression' library which is a bit complicated
import requests
import pandas as pd
import re

# we need somewhere to store our collection of data so create an empty data frame to start
big_df = pd.DataFrame()

# create a loop which will go year-by-year for these years
for year in [2018,2019,2020]:
    # for each year we need to get each month (note Python is *exclusive* here hence 1-13)
    for month in range(1,13):
        # download the page of links to tables for each month of each year published on CAA website
        stub = "https://www.caa.co.uk/Data-and-analysis/UK-aviation-market/Airports/Datasets/UK-Airport-data/"
        # handle the fact that the pages for Jan to Sep include a 0 in the address i.e. the month is padded
        if month < 10:
            padded_month = "0" + str(month)
        else:
            padded_month = str(month)
        target = "Airport-data-" + str(year) + "-" + padded_month + "/"
        url = stub + target
        # print the URL on the screen to show each page we are going to download (helps with spotting any problems)
        print(url)
        # now let's download this page
        html = requests.get(url).text
        # this gives us a big block of HTML, hiding in it is the link to the CSV file we want to get
        # first split the big block into individual lines "\n" here means end of line
        for line in html.split("\n"):
            # this is where we use 'regular expressions' which are a bit complicated
            # an added complication is that for some reason the October file for one year is in a different place!?
            # the next two lines of code basically find the link to the Table_13.csv file that we want to get
            p = 'uploadedFiles\/CAA\/Content\/Standard_Content\/Data_and_analysis\/Datasets\/Airport_stats\/[Airport_data_20.._..|October][^\/]*\/Table_13[^\.]*.csv'
            href = re.search(p, line, flags=re.I)
            # this block of code is what happens when we have found the link
            if href:
                # print the link on the screen to show us what we are trying to download
                print(href.group())
                # download this CSV file into a pandas dataframe using pd.read_csv
                df = pd.read_csv('https://www.caa.co.uk/' + href.group())
                # is our big dataframe empty? if so we can just copy across this data straight into it
                if big_df.empty:
                    big_df = df.copy()
                # if big dataframe is *not* empty we don't want to accidentally overwrite the data already in it!!
                # so *append* the new data to the end of the big dataframe instead
                else:
                    big_df = big_df.append(df)

# finished!
print("Done!")

In [ ]:
# save our data to a new CSV file
# pandas adds an 'index' column that we do not want, using index=False throws it away
big_df.to_csv('cargo_2018-2020.csv', index=False)

In [ ]:
# re-use above code to get Table 09 instead
big_df = pd.DataFrame()
for year in [2018,2019,2020]:
    for month in range(1,13):
        #print(year, month)
        stub = "https://www.caa.co.uk/Data-and-analysis/UK-aviation-market/Airports/Datasets/UK-Airport-data/"
        target = f"Airport-data-{year}-0{month}/" if month < 10 else f"Airport-data-{year}-{month}/"
        url = stub + target
        print(url)
        html = requests.get(url).text
        #print(html)
        for line in html.split("\n"):
            #print(line)
            p = '\/uploadedFiles\/CAA\/Content\/Standard_Content\/Data_and_analysis\/Datasets\/Airport_stats\/[Airport_data_20.._..|October][^\/]*\/Table_09[^\.]*.csv'
            href = re.search(p, line, flags=re.I)
            if href:
                print(href.group())
                df = pd.read_csv('https://www.caa.co.uk' + href.group())
                if big_df.empty:
                    big_df = df.copy()
                else:
                    big_df = big_df.append(df)
                break
print("Done!")   

In [ ]:
# save as a new CSV file
big_df.to_csv('pax_2018-2020.csv', index=False)